# Chapter 4.2 Distance Metrics and Categorical Variables



The distance metrics that we studied in the previous section were designed for quantitative variables. But most data sets will contain a mix of categorical and quantitative variables. Let's return to the Titanic data set and see if we can measure similarity between passengers.

In [23]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.display.max_rows = 5

titanic = pd.read_csv("/data301/data/titanic.csv", sep=",")
titanic

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN
1308,3,0,"Zimmerman, Mr. Leo",male,29.0000,0,0,315082,7.8750,NaN,S,NaN,NaN,NaN


## Converting Categorical Variables to Quantitative Variables

Binary categorical variables (categorical variables with two levels) can be converted into quantitative variables by coding one category as 1 and the other category as 0. But what about a categorical variable with more than 2 categories, like `embarked`, which has 3 categories?

We can convert categorical variables with $K$ categories to $K$ separate **0/1 variables**, or **dummy variables**, each of which is an indicator for a particular category. Although this can be done manually, the easiest way is to use the `pandas` function `get_dummies()`.

In [12]:
pd.get_dummies(titanic["embarked"])

,C,Q,S
0,0,0,1
1,0,0,1
...,...,...,...
1307,1,0,0
1308,0,0,1


Since each observation is in exactly one category, each row contains exactly one 1; the rest are 0s.

We can call `get_dummies` on a `DataFrame` to apply dummy-encoding to multiple variables at once. `pandas` will only dummy-encode the variables it thinks are categorical, so it is necessary to convert any categorical variables that are represented using numeric values. `pandas` will also automatically add the variable name as a prefix to prevent collisions between category names.

In [17]:
# Convert pclass to a categorical type
titanic["pclass"] = titanic["pclass"].astype(str)

# Pass all variables to get_dummies, except ones that are "other" types
titanic_num = pd.get_dummies(
    titanic.drop(["name", "ticket", "cabin", "boat", "body"], axis=1)
)
titanic_num

,survived,age,sibsp,parch,fare,pclass_1,pclass_2,pclass_3,sex_female,sex_male,...,"home.dest_Wimbledon Park, London / Hayling Island, Hants","home.dest_Windsor, England New York, NY","home.dest_Winnipeg, MB","home.dest_Winnipeg, MN","home.dest_Woodford County, KY","home.dest_Worcester, England","home.dest_Worcester, MA","home.dest_Yoevil, England / Cottage Grove, OR","home.dest_Youngstown, OH","home.dest_Zurich, Switzerland"
0,1,29.0000,0,0,211.3375,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.9167,1,2,151.5500,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,0,27.0000,0,0,7.2250,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1308,0,29.0000,0,0,7.8750,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


Now that we have a `DataFrame` of all numeric values, we can apply the methods from the previous section to calculate distances between observations. For example, to find the passenger who is most similar to the first passenger, Elisabeth Watson, we can find the row with the smallest Euclidean distance to that row in the above `DataFrame`.

In [27]:
titanic_std = (titanic_num - titanic_num.mean()) / titanic_num.std()
np.sqrt(((titanic_std - titanic_std.loc[0]) ** 2).sum(axis=1)).sort_values()

0       0.000000
238     1.509375
         ...    
472    41.186785
511    41.209435
dtype: float64

The most similar passenger to Elisabeth Allen, other than herself, is passenger 238. Let's extract these passengers from the original `DataFrame` to see how similar they really are.

In [26]:
titanic.loc[[0, 238]]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
238,1,1,"Robert, Mrs. Edward Scott (Elisabeth Walton Mc...",female,43.0,0,1,24160,211.3375,B3,S,2,NaN,"St Louis, MO"


The two passengers are indeed very similar, only differing in age and the number of parents/children accompanying her. They even happen to share the same first two names ("Elizabeth Walton").

# Exercises

The following exercise uses the Ames housing data set (`/data301/data/AmesHousing.txt`).

**Exercise 1.** Suppose that you really like the first house in the data set, but it is too expensive. Find homes that are similar to it that are cheaper, by calculating distances after encoding categorical variables as dummy variables. Be sure to actually look at the profiles of the homes that your algorithm picked out as most similar. Do they make sense?

Try different distance metrics and different standardization methods. How sensitive are your results to these choices?

_Think:_ If your goal is to find a "good deal" on a similar house, should you include sale price as a variable in your distance metric? 

_Hint:_ There are too many variables in the data set. You will want to pare it down to a manageable number, but be sure to include a mixture of categorical and quantitative variables. Refer to the [data documentation](https://ww2.amstat.org/publications/jse/v19n3/decock/DataDocumentation.txt) for information about the variables.

In [ ]:
# ENTER YOUR CODE HERE